In [1]:
from typing import List
import re
import pdfplumber
import PyPDF2
from unidecode import unidecode
import unicodedata
import spacy
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download("punkt")
from IPython.display import Markdown, display, Video

assesment_draw_and_fill = True

/home/jmaronasm/.conda/envs/GPFLOW/lib/python3.7/site-packages/pdfminer/pdfdocument.py:22: CryptographyDeprecationWarning: Python 3.7 is no longer supported by the Python core team and support for it is deprecated in cryptography. A future release of cryptography will remove support for Python 3.7.
  from cryptography.hazmat.backends import default_backend
2025-03-10 17:35:23.708282: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-10 17:35:23.708319: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package punkt to /home/jmaronasm/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Disclaimer:** This notebook pretends to be didactic, introducing the ideas behind the concepts used to train neural probabilistic language models. This means that, modern models are implemented using advanced algorithms based on these ideas.

# Dataset

Let's start by reading data from memory. In this case we will be reading data from pdf but this similar proceedure might apply to other sources such as webpages.

## Pdf scrapping

Let's practice scrapping data from a, typically, unstructured data source such as a pdf or a webpage. In this case we will be reading data from a pdf that only contains text, using  pdfplumber and PyPDF2 packages. As you'll see, there are pdf extractors that are more advanced than others.

Note that when you have some text such as text with an underscore, this underscore is codified by your pdf using a special character. The pdf renderer interprets this special character and the graphical interface draws the line, but in memory this is just a byte representing some character.

PyPDF2 will decode this character into whatever something which is undesirable. You can note how using PyPDF2 will read the source text $\underline{\text{Conducción autónoma (Tesla):}}$ into $\text{Conducción autónoma (T esla):}$ while pdfplumber will work as expected.



##### Task: Implement data extraction with PyPDF2.

In [2]:
if assesment_draw_and_fill:
    code = """
```python
raw_data = ...

## reading dataset
with open(..., 'rb') as file:
    ... = PyPDF2.PdfReader(...)
    # Extract text from each page
    for ... in pdf_reader.pages:
        text = ....extract_text()
        raw_data ...

## visualize the raw readed data.
print(raw_data)
```
"""

    display(Markdown(code))


```python
raw_data = ...

## reading dataset
with open(..., 'rb') as file:
    ... = PyPDF2.PdfReader(...)
    # Extract text from each page
    for ... in pdf_reader.pages:
        text = ....extract_text()
        raw_data ...

## visualize the raw readed data.
print(raw_data)
```


In [3]:
raw_data = ""

## reading dataset
with open('../data/texto_pdf.pdf', 'rb') as file:
    pdf_reader = PyPDF2.PdfReader(file)
    # Extract text from each page
    for page in pdf_reader.pages:
        text = page.extract_text()
        raw_data += text

## visualize the raw readed data.
print(raw_data)

REDES NEURONALES
Redes Neuronales y Cómo Funcionan
Inspiración en el cerebro:  Las redes neuronales están inspiradas en 
cómo funciona el cerebro humano. Nuestro cerebro, específicamente 
el área visual, recibe información visual (como lo que vemos) y 
empieza por detectar cosas simples como líneas o bordes. Luego, va 
construyendo cosas más complejas, como formas, objetos, y 
finalmente, cosas que reconocemos (por ejemplo, una cara o un 
coche).
En las redes neuronales artificiales  pasa algo parecido: las primeras 
"capas" de la red se encargan de cosas básicas como detectar líneas, 
y las capas más profundas hacen un trabajo más complejo, como 
identificar qué es lo que la imagen representa.
Capas de una red:  Imagina que cada capa de la red es como una 
persona en una cadena de montaje. Cada persona se especializa en 
hacer una cosa: las primeras solo detectan si hay líneas, otras más 
adelante se encargan de juntar esas líneas y formar esquinas o 
curvas, y las últimas capas se en

##### Task: Implement data extraction with pdfplumber.

In [4]:
if assesment_draw_and_fill:
    code = """
```python
raw_data = ...

## reading dataset with PDF Plumber
with pdfplumber.open(...) as pdf:
    for ... in pdf.pages:
        ... += page.extract_text()

## visualize the raw readed data.
print(...)
```
"""

    display(Markdown(code))


```python
raw_data = ...

## reading dataset with PDF Plumber
with pdfplumber.open(...) as pdf:
    for ... in pdf.pages:
        ... += page.extract_text()

## visualize the raw readed data.
print(...)
```


In [6]:
raw_data = ""

## reading dataset with PDF Plumber
with pdfplumber.open('../data/texto_pdf.pdf') as pdf:
    for page in pdf.pages:
        raw_data += page.extract_text()

## visualize the raw readed data.
print(raw_data)

REDES NEURONALES
Redes Neuronales y Cómo Funcionan
Inspiración en el cerebro: Las redes neuronales están inspiradas en
cómo funciona el cerebro humano. Nuestro cerebro, específicamente
el área visual, recibe información visual (como lo que vemos) y
empieza por detectar cosas simples como líneas o bordes. Luego, va
construyendo cosas más complejas, como formas, objetos, y
finalmente, cosas que reconocemos (por ejemplo, una cara o un
coche).
En las redes neuronales artificiales pasa algo parecido: las primeras
"capas" de la red se encargan de cosas básicas como detectar líneas,
y las capas más profundas hacen un trabajo más complejo, como
identificar qué es lo que la imagen representa.
Capas de una red: Imagina que cada capa de la red es como una
persona en una cadena de montaje. Cada persona se especializa en
hacer una cosa: las primeras solo detectan si hay líneas, otras más
adelante se encargan de juntar esas líneas y formar esquinas o
curvas, y las últimas capas se encargan de decirt

##### Question: What differences do you observe?

Obviously, this is a noticeable difference. And is something in big pdf files hard to spot. This is probably the day by day of a NLP engineer. Your experience and a good search of different options to read pdf before implementation is the key difference. ChatGPT is something might be very usefull for this comparison (this is actually how I have arrived at pdfplumber since the one I knew and used was PyPDF2).

## Text processing and tokenization

Note that, depending on your particular application, we need to apply some text processing before training your model. It is quite reasonable to assume that, if we want just to implement a simple text generator, accents, capital letters and so on are vacuous for our application. In a real application, we have enough training data to have specific tokens for capital or accents. Actually tokens in modern applications do not use word level tokenizer which is the one we will be using but a more advanced one (named Byte pair encoding). 

Byte pair encodings work by defining tokens on subwords, and this naturally handle accents, capital letters and so on without having to have a token for word "Hola" and "hola" (depending on if the word is the beginning of a sentence or not).

Note however, that you can always post process the output of your model so that when your model outputs a dot ".", then you know the next word should go in capitals. So you can basically get the next generated word and process it to be in capital, without needing the model to know this. Obviously with enough data and compute your model will always learn to do this.

Also, if your goal is just to extract semantic meaning from sentences or words, then the more redundant information you remove the better, and obviously capital letters is not important.


In other words, Natural Language modelling is a mix of preprocessing, probabilistic modelling, and postprocessing, where the way you treat these different aspects differ on the application you want to achieve.


### Text processing / normalization.

Text processing is also known as normalization, and is the step you perform to your data before starting its numerical representation (tokenization). As mentioned in the previous section, it will depend on whatever task you would like to do. For semantic embeddings, for example, there is no need to keep capital words because they usually have very few relevant semantic meaning, if this semantic meaning is going to be used to, for example, training a classifier to discriminate between racist and non racist content.


##### Task: 
Implement a text processing routine that remove accents and put everything into lower letters. You could use lower() and unidecode. What is the problem with unidecode?. You should apply this routine to the readed data from the pdf in the previous step, which is saved in the variable `raw_data`. Be aware of using the one readed with pdf plumber.

To test unidecode first do it in the following sentence:

```python
x = "Júan Maroñas françois"
```

In [7]:
if assesment_draw_and_fill:
    code = """
```python
## Test unidecode with accents and non standard ascii characters.
print(unidecode("Júan Maroñas françois"))
```
"""

    display(Markdown(code))


```python
## Test unidecode with accents and non standard ascii characters.
print(unidecode("Júan Maroñas françois"))
```


In [8]:
## Test unidecode with accents and non standard ascii characters.
print(unidecode("Júan Maroñas françois"))

Juan Maronas francois


In [9]:
if assesment_draw_and_fill:
    code = """
```python
## make lower
...
## make closest non ascii character into ascii character. Basically it removes
...

print(proc_data)
```
"""

    display(Markdown(code))


```python
## make lower
...
## make closest non ascii character into ascii character. Basically it removes
...

print(proc_data)
```


In [10]:
## make lower
proc_data = raw_data.lower()
## make closest non ascii character into ascii character. Basically it removes
proc_data = unidecode(proc_data)

print(proc_data)

redes neuronales
redes neuronales y como funcionan
inspiracion en el cerebro: las redes neuronales estan inspiradas en
como funciona el cerebro humano. nuestro cerebro, especificamente
el area visual, recibe informacion visual (como lo que vemos) y
empieza por detectar cosas simples como lineas o bordes. luego, va
construyendo cosas mas complejas, como formas, objetos, y
finalmente, cosas que reconocemos (por ejemplo, una cara o un
coche).
en las redes neuronales artificiales pasa algo parecido: las primeras
"capas" de la red se encargan de cosas basicas como detectar lineas,
y las capas mas profundas hacen un trabajo mas complejo, como
identificar que es lo que la imagen representa.
capas de una red: imagina que cada capa de la red es como una
persona en una cadena de montaje. cada persona se especializa en
hacer una cosa: las primeras solo detectan si hay lineas, otras mas
adelante se encargan de juntar esas lineas y formar esquinas o
curvas, y las ultimas capas se encargan de decirt

As you can see unidecode replaces a non ascii character with the closest ascii character. In our example it remove letters with accents but also removes french letter "ç" or spanish letter "ñ". This is obviously undesirable. 

How can we replace accents letters with standard letters?. Using a regex and the module unidecodedata.

Creating regex with chatGPT is very simple and is a good usage of this tool. So ask chatGPT to provide a regex expression to remove any type of accents and start looping with its output.

As you'll see it will probably first ask you to use the unicodedata module (that btw is recommended in other resources when googling `https://www.geeksforgeeks.org/how-to-remove-string-accents-using-python-3/`) to replace the text with accent by the ascii character and then its accent. So that later using a regex expression we can remove accents and finally join everything to recover back the text with the spanish n or other desired letters.

Btw, do not trust chatGPT by itself and always be critic on its output. For example the solution it gave me was to use the following regular expression:

```
'(?<!u)[\u0301\u0300\u0302\u0303]'
```

After I saw this I check what this regular expression is asking for, and found out that the only regular expression I need is:

```
'[\u0301\u0300\u0302]'
```

Test both in the following cell and inspect differences.

As before, first check your regular expressions on:

```python
x = "Juan Maroñas ácento diëresis"
```

In [13]:
if assesment_draw_and_fill:
    code = """
```python
text = "Juan Maroñas ácento diëresis"

proc_text = unicodedata.normalize("NFD", text)
proc_text = re.sub(r"[\\u0301\\u0300\\u0302]", '', proc_text)
proc_text = unicodedata.normalize("NFC", proc_text)

print(proc_text)
```
"""

    display(Markdown(code))


```python
text = "Juan Maroñas ácento diëresis"

proc_text = unicodedata.normalize("NFD", text)
proc_text = re.sub(r"[\u0301\u0300\u0302]", '', proc_text)
proc_text = unicodedata.normalize("NFC", proc_text)

print(proc_text)
```


In [14]:
text = "Juan Maroñas ácento diëresis"


proc_text = unicodedata.normalize("NFD", text)
proc_text = re.sub(r'[\u0301\u0300\u0302]', '', proc_text)
proc_text = unicodedata.normalize("NFC", proc_text)

print(proc_text)

Juan Maroñas acento diëresis


##### Task: 
Modify your text processing routine to remove accents and move everything into lower using these regular expressions instead of unidecoding.


In [16]:
if assesment_draw_and_fill:
    code = """
```python
## make lower
proc_data = raw_data.lower()

## remove accents
proc_data = unicodedata.normalize("NFD", proc_data)
proc_data = re.sub(r'[\\u0301\\u0300\\u0302]', '', proc_data)
proc_data = unicodedata.normalize("NFC", proc_data)
```
"""

    display(Markdown(code))


```python
## make lower
proc_data = raw_data.lower()

## remove accents
proc_data = unicodedata.normalize("NFD", proc_data)
proc_data = re.sub(r'[\u0301\u0300\u0302]', '', proc_data)
proc_data = unicodedata.normalize("NFC", proc_data)
```


In [17]:
## make lower
proc_data = raw_data.lower()

## remove accents
proc_data = unicodedata.normalize("NFD", proc_data)
proc_data = re.sub(r'[\u0301\u0300\u0302]', '', proc_data)
proc_data = unicodedata.normalize("NFC", proc_data)

print(proc_data)

redes neuronales
redes neuronales y como funcionan
inspiracion en el cerebro: las redes neuronales estan inspiradas en
como funciona el cerebro humano. nuestro cerebro, especificamente
el area visual, recibe informacion visual (como lo que vemos) y
empieza por detectar cosas simples como lineas o bordes. luego, va
construyendo cosas mas complejas, como formas, objetos, y
finalmente, cosas que reconocemos (por ejemplo, una cara o un
coche).
en las redes neuronales artificiales pasa algo parecido: las primeras
"capas" de la red se encargan de cosas basicas como detectar lineas,
y las capas mas profundas hacen un trabajo mas complejo, como
identificar que es lo que la imagen representa.
capas de una red: imagina que cada capa de la red es como una
persona en una cadena de montaje. cada persona se especializa en
hacer una cosa: las primeras solo detectan si hay lineas, otras mas
adelante se encargan de juntar esas lineas y formar esquinas o
curvas, y las ultimas capas se encargan de decirt

##### Task: Finally, use regular expressions to remove more than one blank space and more than one separate line. In general to remove any separator token such as \t, \n etc and replace it by a blank space.

You can ask GPT for a regular expression to do this for you.

Apply this regular expression to the data you have been processing in the step before.

Before doing that, test it on:

```python
text = "HOOOla me       llamo \t juan    \n.\n tu como te llamas ? \n Manolo"
```

In [18]:
if assesment_draw_and_fill:
    code = """
```python
## remove more than one blank space
text = '''HOOOla me       llamo \t juan    \n.\n tu como te llamas ? \n Manolo'''

print(text)

text = re.sub(..., ..., ...)

print(text)
```
"""

    display(Markdown(code))


```python
## remove more than one blank space
text = '''HOOOla me       llamo 	 juan    
.
 tu como te llamas ? 
 Manolo'''

print(text)

text = re.sub(..., ..., ...)

print(text)
```


In [19]:
## remove more than one blank space
text = "HOOOla me       llamo \t juan    \n.\n tu como te llamas ? \n Manolo"

print(text)

text = re.sub(r'\s+', ' ', text)

print(text)

HOOOla me       llamo 	 juan    
.
 tu como te llamas ? 
 Manolo
HOOOla me llamo juan . tu como te llamas ? Manolo


In [20]:
if assesment_draw_and_fill:
    code = """
```python
proc_data = ...

print(proc_data)
```
"""

    display(Markdown(code))


```python
proc_data = ...

print(proc_data)
```


In [21]:
proc_data = re.sub(r'\s+', ' ', proc_data)

print(proc_data)

redes neuronales redes neuronales y como funcionan inspiracion en el cerebro: las redes neuronales estan inspiradas en como funciona el cerebro humano. nuestro cerebro, especificamente el area visual, recibe informacion visual (como lo que vemos) y empieza por detectar cosas simples como lineas o bordes. luego, va construyendo cosas mas complejas, como formas, objetos, y finalmente, cosas que reconocemos (por ejemplo, una cara o un coche). en las redes neuronales artificiales pasa algo parecido: las primeras "capas" de la red se encargan de cosas basicas como detectar lineas, y las capas mas profundas hacen un trabajo mas complejo, como identificar que es lo que la imagen representa. capas de una red: imagina que cada capa de la red es como una persona en una cadena de montaje. cada persona se especializa en hacer una cosa: las primeras solo detectan si hay lineas, otras mas adelante se encargan de juntar esas lineas y formar esquinas o curvas, y las ultimas capas se encargan de decirt

##### Task: To finish thes text normalization step, implement, in a single cell, all the steps you have performed:

* reading data from source.
* lowering
* removing accents
* removing separator spaces

In [23]:
## variable where content is readed into
raw_data = ""

## reading dataset with PDF Plumber
with pdfplumber.open('../data/texto_pdf.pdf') as pdf:
    for page in pdf.pages:
        raw_data += page.extract_text()

## make lower
proc_data = raw_data.lower()

## remove accents
proc_data = unicodedata.normalize("NFD", proc_data)
proc_data = re.sub(r'[\u0301\u0300\u0302]', '', proc_data)
proc_data = unicodedata.normalize("NFC", proc_data)

## remove more than one separator into blank space
proc_data = re.sub(r'\s+', ' ', proc_data)

print(proc_data)

redes neuronales redes neuronales y como funcionan inspiracion en el cerebro: las redes neuronales estan inspiradas en como funciona el cerebro humano. nuestro cerebro, especificamente el area visual, recibe informacion visual (como lo que vemos) y empieza por detectar cosas simples como lineas o bordes. luego, va construyendo cosas mas complejas, como formas, objetos, y finalmente, cosas que reconocemos (por ejemplo, una cara o un coche). en las redes neuronales artificiales pasa algo parecido: las primeras "capas" de la red se encargan de cosas basicas como detectar lineas, y las capas mas profundas hacen un trabajo mas complejo, como identificar que es lo que la imagen representa. capas de una red: imagina que cada capa de la red es como una persona en una cadena de montaje. cada persona se especializa en hacer una cosa: las primeras solo detectan si hay lineas, otras mas adelante se encargan de juntar esas lineas y formar esquinas o curvas, y las ultimas capas se encargan de decirt

### Text tokenization.

As you have seen, there are some things missing from our processing. For example highlighted words such as "pesos" has not been considered. Note, how variable a natural language text could be and how complex the step of preprocessing might be. It might be the case that, on huge text, some of these things might not be spotted and processed adequately.

Fortunately, there are some libraries that have already implemented all these considerations, in objects usually known as tokenizers. This tokenizers are in charge of both text normalization and text tokenization, which is the concept I'll explained know.

Tokenization is the process of getting raw text, and decide which are the fundamental units that will represent this text. For example the words of a language and punctuation signs could be these individual units. Thus a word with and without capital or accents is a different unit. So `hola` and `Hola` and `HOla` are different tokens attending to this definition. Moreover, words like `"hola"` is also different from `hola?` or `hola:`

This is why normalizing the text before tokenization is so important, otherwise we will be creating extra token representations that do not provide additional information (obviously this depends on the application and amount of training data. If you have enough training data then is just fine). Note that it is better to have a token for each punctuation sign `.` and word `hola` and use both of them to represent `hola.` than have one extra token for each combination of `hola` with any possible punctuation sign, since that needs to be done for each word in the vocabulary.

We will later see in this series how post processing can actually get the output of a model and transform it in the way we actually write, since when we write a punctuation sign we do it right next to the word, and not putting a blank space in between. But why should a machine laerning model learn to do this when we can just actually code it up deterministically?. We will see this later.

Thus, the process of text preprocessing is in charge of removing redundance information in order to get the word representations we care about. Note that, if the previous text get splitted by blank spaces, we will get one token being "learning," and another will be "learning" in the same way we will have "numero," vs "numero" or "principio," vs "principio". However, it looks more natural to have a token for "learning" a token for "numero" and a token for "," which could be something like "<\comma>". So what does a tokenizer do? Well a tokenizer prepocess text in a way to remove some of these redudances, for example removing accents (if we require so) or splitting words by "," and transform our splitted text into the unique tokens, labelling each token with a number. This is known as the vocabulary and is represented through $V$, with $\mid V\mid$ being the total number of elements in the vocabulary.

Libraries do this, one of the most popular is nltk. I have checked torchtext have some tokenizers implemented, such as the GPTBPE, or the Bert tokenizer, and also allow us to use tokenizer from Huggingface. For an overview of modern tokenizers see: `https://huggingface.co/docs/tokenizers/pipeline`. As you see here a tokenizer is divided into:

* normalization step.
* a token to id step. This requirese a pretokenization (for example splitting text into words and then applying a set operation) and training the tokenizer (just in some cases. We could se the fact of creating a hash table that maps tokens to token's_ids as these process of tokenization)
* post processing: adding special tokens that are not part of the vocabulary, such as a begin of sequence token, padding token, end of sequence token, separation token ...

We will create a word level tokenizer using words as individuals. However, modern tokenizers work at subwords. This makes a lot of sense. Note that a word level tokenizer will use a different token for plural and singular words. However if you tokenize at subwords then yo do not have this problem, because words will share lot's of subwords. These tokens are usually learnt automatically from data. 

In modern models, tokens are usually learned from data and represent subwords rather than words, such as the Byte Pair Encoding (Developed by openAI and opensource) or the WordPiece (Developed by google and closed source) tokenizers.

##### Task:

Let's use NLTK to tokenize our text using words and punctuation sings as the fundamental tokens. There are lots of punctuation signs such as !,¿,? etc and splitting by all of them using python string split method might be tedious because we might forget some of them, so let's do it with a library. Note that, depending on the text application, we will like to split by characters like "%" "#" "/" or "$". We need to test this otherwise we will need a regular expression.

Tokenize your data using the word_tokenize function from nltk library. Instead of working with the normalized text you read from the previous part of the assessment, use the sentence:

```python
x =  """¡Hola! ¿Cómo estás? Me debes 50€ al 50% TAE y mi dial es #234, y además eres \"muy_tonto\" y te gusta la ñ de españa? ni los pingüinos."""

```


In [24]:
if assesment_draw_and_fill:
    code = """
```python
texto = "¡Hola! ¿Cómo estás? Me debes 50€ al 50% TAE y mi dial es #234, y además eres \"muy_tonto\" y te gusta la ñ de españa? ni los pingüinos."

tokens = ...

print(tokens)
```
"""

    display(Markdown(code))


```python
texto = "¡Hola! ¿Cómo estás? Me debes 50€ al 50% TAE y mi dial es #234, y además eres "muy_tonto" y te gusta la ñ de españa? ni los pingüinos."

tokens = ...

print(tokens)
```


In [25]:
texto = """¡Hola! ¿Cómo estás? Me debes 50€ al 50% TAE y mi dial es #234, y además eres \"muy_tonto\" y te gusta la ñ de españa? ni los pingüinos."""

tokens = word_tokenize(texto)

print(tokens)

['¡Hola', '!', '¿Cómo', 'estás', '?', 'Me', 'debes', '50€', 'al', '50', '%', 'TAE', 'y', 'mi', 'dial', 'es', '#', '234', ',', 'y', 'además', 'eres', '``', 'muy_tonto', "''", 'y', 'te', 'gusta', 'la', 'ñ', 'de', 'españa', '?', 'ni', 'los', 'pingüinos', '.']


As you can see it is not a good tokenizer for the spanish language because it does not take into consideration that we use symbols such as "¡,¿" to begin our sentences or special characters such as dolar or euros. 

Attending to nltk documentation, `word_tokenize` accepts a language argument to specify language.

##### Task: Pass in the argument `language = 'spanish'` to the word_tokenize function and see if it i solved

In [26]:
texto = """¡Hola! ¿Cómo estás? Me debes 50€ al 50% TAE y mi dial es #234, y además eres \"muy_tonto\" y te gusta la ñ de españa? ni los pingüinos."""

tokens = word_tokenize(texto, language = 'spanish')

print(tokens)

['¡Hola', '!', '¿Cómo', 'estás', '?', 'Me', 'debes', '50€', 'al', '50', '%', 'TAE', 'y', 'mi', 'dial', 'es', '#', '234', ',', 'y', 'además', 'eres', '``', 'muy_tonto', "''", 'y', 'te', 'gusta', 'la', 'ñ', 'de', 'españa', '?', 'ni', 'los', 'pingüinos', '.']


It does not work well aswell. A good alternative to nltk is spacy, which is another natural language toolkit.

##### Task: tokenize using spacy.

In [27]:
if assesment_draw_and_fill:
    code = """
```python
texto =  "¡Hola! ¿Cómo estás? Me debes 50€ al 50% TAE y mi dial es #234, y además eres \"muy_tonto\" y te gusta la ñ de españa? ni los pingüinos.

spacy_es_tokenizer = spacy.load('es_core_news_sm')

tokens = spacy_es_tokenizer(texto)
tokens = [t for t in tokens] 
print(tokens)
```
"""

    display(Markdown(code))


```python
texto =  "¡Hola! ¿Cómo estás? Me debes 50€ al 50% TAE y mi dial es #234, y además eres "muy_tonto" y te gusta la ñ de españa? ni los pingüinos.

spacy_es_tokenizer = spacy.load('es_core_news_sm')

tokens = spacy_es_tokenizer(texto)
tokens = [t for t in tokens] 
print(tokens)
```


In [28]:
texto =  """¡Hola! ¿Cómo estás? Me debes 50€ al 50% TAE y mi dial es #234, y además eres \"muy_tonto\" y te gusta la ñ de españa? ni los pingüinos."""

spacy_es_tokenizer = spacy.load('es_core_news_sm')

tokens = spacy_es_tokenizer(texto)
tokens = [t for t in tokens] 
print(tokens)


[¡, Hola, !, ¿, Cómo, estás, ?, Me, debes, 50, €, al, 50%, TAE, y, mi, dial, es, #, 234, ,, y, además, eres, ", muy_tonto, ", y, te, gusta, la, ñ, de, españa, ?, ni, los, pingüinos, .]


However, we still see that some tokens such as % are not correctly splitted. How can we do this? Well with a regular expression. What we need is a regular expression that matches any possible alfanumeric character plus "_" (which is what "\w" means as a regular expression) or matches any string that does not match "\w" or any separator such as blank spaces, new lines etc, which is represented by "s". 

##### Task: Implement a regular expression and apply it to the same sentence. This regular expression must:

* split sentences by blank spaces into words.
* If a word is with any symbol that is not a letter or a number, the symbol should be splitter individually.

Tips: 
* `\w` in a regular expression matches any alfanumeric character (letters plus numbers) and symbol "_". 
* `\s` matches any separation.
* symbol `+` indicates at least one match or more. So `\w+` matches any alfanumeric character followed by alfanumeric characters, until it finds something it is not.
* `^` negates. So it basically finds the elements that do not match with that expression. 

Remember you can ask chatGPT to see what it tells you. Tell it you want to have a regular expression that splits by words, punctuation signs and other simbols. You can also check how the word split tokenizer at huggingface works, which is what I have done. Basically look at its source code.

In [29]:
if assesment_draw_and_fill:
    code = """
```python
texto = "¡Hola! ¿Cómo estás? Me debes 50€ al 50% TAE y mi dial es #234, y además eres \"muy_tonto\" y te gusta la ñ de españa? ni los pingüinos.

tokens = re.findall(r'...',...)

print(tokens)
```
"""

    display(Markdown(code))


```python
texto = "¡Hola! ¿Cómo estás? Me debes 50€ al 50% TAE y mi dial es #234, y además eres "muy_tonto" y te gusta la ñ de españa? ni los pingüinos.

tokens = re.findall(r'...',...)

print(tokens)
```


In [30]:
texto = """¡Hola! ¿Cómo estás? Me debes 50€ al 50% TAE y mi dial es #234, y además eres \"muy_tonto\" y te gusta la ñ de españa? ni los pingüinos."""

tokens = re.findall(r'\w+|[^\w\s]+',texto)


print(tokens)

['¡', 'Hola', '!', '¿', 'Cómo', 'estás', '?', 'Me', 'debes', '50', '€', 'al', '50', '%', 'TAE', 'y', 'mi', 'dial', 'es', '#', '234', ',', 'y', 'además', 'eres', '"', 'muy_tonto', '"', 'y', 'te', 'gusta', 'la', 'ñ', 'de', 'españa', '?', 'ni', 'los', 'pingüinos', '.']


As you see by using `\w` we are not matching "_" as a token. To do so we can specify all the characters we want to match. This is done by:

```python
texto = """¡Hola! ¿Cómo estás? Me debes 50€ al 50% TAE y mi dial es #234, y además eres \"muy_tonto\" y te gusta la ñ de españa? ni los pingüinos."""

tokens = re.findall(r'[â-ûÂ-Ûä-üÄ-Üà-ùÀ-Ùá-úÁ-Úa-zA-Z0-9ñÑçÇ]+|[^â-ûÂ-Ûä-üÄ-Üà-ùÀ-Ùá-úÁ-Úa-zA-Z0-9ñÑçÇ]+',texto)

print(tokens)
```

In [31]:
texto = """¡Hola! ¿Cómo estás? Me debes 50€ al 50% TAE y mi dial es #234, y además eres \"muy_tonto\" y te gusta la ñ de españa? ni los pingüinos."""

tokens = re.findall(r'[â-ûÂ-Ûä-üÄ-Üà-ùÀ-Ùá-úÁ-Úa-zA-Z0-9ñÑçÇ]+|[^â-ûÂ-Ûä-üÄ-Üà-ùÀ-Ùá-úÁ-Úa-zA-Z0-9ñÑçÇ]+',texto)

print(tokens)

['¡', 'Hola', '! ¿', 'Cómo', ' ', 'estás', '? ', 'Me', ' ', 'debes', ' ', '50', '€ ', 'al', ' ', '50', '% ', 'TAE', ' ', 'y', ' ', 'mi', ' ', 'dial', ' ', 'es', ' #', '234', ', ', 'y', ' ', 'además', ' ', 'eres', ' "', 'muy', '_', 'tonto', '" ', 'y', ' ', 'te', ' ', 'gusta', ' ', 'la', ' ', 'ñ', ' ', 'de', ' ', 'españa', '? ', 'ni', ' ', 'los', ' ', 'pingüinos', '.']


This solves the problem in a unique regular expression. It is then fast, but not very clean. Also, it is very prone to errors because we might forget possible alfanumeric characters or letters in some language, but we obviously require to check the ascii codification table  `https://www.ascii-code.com/`. 

A more elegant solution yet being slower is to first split by character "_" and then split using the previous regular expression. So do a past through all the text splitting by that symbol, and then another pass using the above regular expression:

```python
texto = """¡Hola! ¿Cómo estás? Me debes 50€ al 50% TAE y mi dial es #234, y además eres \"muy_tonto\" y te gusta la ñ de españa? ni los pingüinos."""

_tokens = re.findall(r'_|[^_]+', texto)
print(_tokens)

tokens = []
for t in _tokens:
    tokens.extend(re.findall(r'\w+|[^\w\s]+',t))

print(tokens)
```

In [32]:
texto = """¡Hola! ¿Cómo estás? Me debes 50€ al 50% TAE y mi dial es #234, y además eres \"muy_tonto\" y te gusta la ñ de españa? ni los pingüinos."""

_tokens = re.findall(r'_|[^_]+', texto)
print(_tokens)

tokens = []
for t in _tokens:
    tokens.extend(re.findall(r'\w+|[^\w\s]+',t))

print(tokens)

['¡Hola! ¿Cómo estás? Me debes 50€ al 50% TAE y mi dial es #234, y además eres "muy', '_', 'tonto" y te gusta la ñ de españa? ni los pingüinos.']
['¡', 'Hola', '!', '¿', 'Cómo', 'estás', '?', 'Me', 'debes', '50', '€', 'al', '50', '%', 'TAE', 'y', 'mi', 'dial', 'es', '#', '234', ',', 'y', 'además', 'eres', '"', 'muy', '_', 'tonto', '"', 'y', 'te', 'gusta', 'la', 'ñ', 'de', 'españa', '?', 'ni', 'los', 'pingüinos', '.']


Note that in doing this kind of tokenizations we would probably require advanced tokenization steps or considerations to, for example, know that tokens corresponding to "#" "234" should be joined in a way and given to the model to represent this fact. For example we could have a token which would represent dial's of the form `#number` or whatever or replace a name by "NOUN", any date in the text by "DATE" and so on. This obviously depends on the application. If your application requires specific date constructions then you need to represent each possible date by `token_number token_slash token_number token_slash token_number` that would represent `24/12/2024`. However if your application just requires knowing there is any possible date, then any date can be repalced by `token_date`, through a regular expression.

The last step is identifying each token with a unique id through a hash table (so that indexing is fast). In python dictionaries are hash tables.

##### Task: create a dictionary whose index is each unique word word and whose value is a unique number. Do it on the tokens generated in the step before

In [33]:
if assesment_draw_and_fill:
    code = """
```python
print(tokens)

vocab = set(tokens)
word_to_id = ...

for i, v in enumerate(vocab):
    ...

print(word_to_id)
```
"""

    display(Markdown(code))


```python
print(tokens)

vocab = set(tokens)
word_to_id = ...

for i, v in enumerate(vocab):
    ...

print(word_to_id)
```


In [34]:
print(tokens)

vocab = set(tokens)
word_to_id = {}

for i, v in enumerate(vocab):
    word_to_id[v] = i

print(word_to_id)

['¡', 'Hola', '!', '¿', 'Cómo', 'estás', '?', 'Me', 'debes', '50', '€', 'al', '50', '%', 'TAE', 'y', 'mi', 'dial', 'es', '#', '234', ',', 'y', 'además', 'eres', '"', 'muy', '_', 'tonto', '"', 'y', 'te', 'gusta', 'la', 'ñ', 'de', 'españa', '?', 'ni', 'los', 'pingüinos', '.']
{'"': 0, 'es': 1, '?': 2, 'ñ': 3, 'pingüinos': 4, '_': 5, 'Hola': 6, 'Me': 7, 'eres': 8, '¿': 9, '.': 10, 'debes': 11, 'además': 12, 'la': 13, '50': 14, 'dial': 15, '234': 16, 'TAE': 17, 'muy': 18, '#': 19, '!': 20, 'españa': 21, 'tonto': 22, 'te': 23, 'gusta': 24, 'de': 25, '€': 26, ',': 27, 'los': 28, 'al': 29, 'estás': 30, '¡': 31, 'y': 32, 'Cómo': 33, 'mi': 34, '%': 35, 'ni': 36}


##### Task:

Now put everything together with our original dataset: normalization plus tokenization.

In [35]:
## make lower
proc_data = raw_data.lower()

## remove accents
proc_data = unicodedata.normalize("NFD", proc_data)
proc_data = re.sub(r'[\u0301\u0300\u0302]', '', proc_data)
proc_data = unicodedata.normalize("NFC", proc_data)

## remove more than one separator into blank space
proc_data = re.sub(r'\s+', ' ', proc_data)

## split into tokens
_tokens = re.findall(r'_|[^_]+', proc_data)

tokens = []
for t in _tokens:
    tokens.extend(re.findall(r'\w+|[^\w\s]+',t))

vocab = set(tokens)
word_to_id = {}

for i, v in enumerate(vocab):
    word_to_id[v] = i

print(word_to_id)
print(len(vocab))

{'ciencia': 0, 'podemos': 1, 'hacerlo': 2, 'el': 3, 'coche': 4, 'capa': 5, 'cosa': 6, 'numero': 7, 'sea': 8, 'que': 9, 'otro': 10, 'gatos': 11, 'resolver': 12, 'ejemplos': 13, 'recibe': 14, 'alta': 15, 'peatones': 16, 'mirar': 17, 'plegamiento': 18, 'sobre': 19, 'lineas': 20, '3': 21, 'situacion': 22, 'trucos': 23, 'le': 24, 'neuronales': 25, ')': 26, 'especificamente': 27, 'fotos': 28, 'encontrar': 29, 'parecido': 30, 'dia': 31, 'especializa': 32, 'al': 33, 'y': 34, 'red': 35, '"': 36, 'juntar': 37, 'datos': 38, 'objetos': 39, 'objetivo': 40, 'sera': 41, 'humano': 42, 'estudiar': 43, 'rompecabezas': 44, 'suficientemente': 45, 'tipico': 46, 'grande': 47, 'entender': 48, 'conducen': 49, 'muy': 50, 'letra': 51, 'reducianlos': 52, 'horizontal': 53, 'se': 54, 'imagina': 55, 'vertical': 56, 'representan': 57, 'informacion': 58, 'alrededor': 59, 'redes': 60, 'verticales': 61, 'una': 62, 'mucho': 63, 'learning': 64, 'va': 65, 'practico': 66, 'usan': 67, 'final': 68, 'trata': 69, 'ajustando': 

# Tokenizer

Let's wrap all these things in a tokenizer class that we can use later on. This is what you used in huggingface or whatever. Remember your tokenizer must be in charge of normalizing and tokenizing. We will also add some usefull methods to convert words to token ids and the opposite.  Note that, in creating this tokenizer you are basically asked to use the code you have been developing during this assement in the different methods proposed:

The structure of your tokenizer must be:

```python
class Tokenizer:
    def __init__(self):
        ...
    
    def encode(self, data:str):
        """Convert raw data to corresponding token list"""
        ...
        return tokens

    def batch_encode(self, data:list):
        """Convert a list containing raw data to a list containing their corresponding token lists."""
        ...
        return tokens

    def decode(self, tokens:list[int]):
        """Given a list of tokens, conver it to a string replacing tokens with text"""
        ...
        return text

    def batch_decode(self, tokens : list[list[int]]):
         """Given a list containing list of tokens, convert it to a list containing strings replacing the tokens with text."""
        ...
        return text
        
    def train_tokenizer(self, raw_data):
        """Given raw data create the token ids by applying the normalization step, pre tokenization step, training step and post tokenization"""
        ...

    def __len__(self):
        """Return the length of the vocabulary"""
        return ...
        
    def _normalize(self, raw_data):
        """Applies normalization steps to raw data"""
        ...
        return proc_data

    def _pre_tokenize(self, proc_data):
        """Apply pre tokenization step to normalized data"""
        ...
        return tokens
```

##### Task: Create tokenizer class.

In [36]:
if assesment_draw_and_fill:
    code = """
```python
class Tokenizer:
    def __init__(self):
        # regular expression to get tokens by words and punctuation signs.
        self.re_get_tokens = re.compile(...)
        # regular expression to replace string splitters such as blank space or new lines etc with whatever
        self.re_replace_string_splitters = re.compile(...)
        # regular expression to split by underscore "_"
        self.split_by_underscore = re.compile(....)
        # word to id dictionary
        self.word_to_ids = one
    
    def encode(self, data:str):
        "Convert raw data to corresponding token list"
        assert self.word_to_ids is not None, "Tokenizer has not been train. Call self.train_tokenizer"

        proc_data = ...
        proc_data = ...

        tokens = []
        for token_text in proc_data:
            tokens.append(...)

        return tokens

    def batch_encode(self, data:list):
        "Convert a list containing raw data to a list containing their corresponding token lists."
        tokens = []
        for d in data:
            tokens.append(self.encode(d))

        return tokens

    def decode(self, tokens:list[int]):
        "Given a list of tokens, conver it to a string replacing tokens with text"
        assert self.word_to_ids is not None, "Tokenizer has not been train. Call self.train_tokenizer"
        
        text = ...
        text = ' '.join(text)

        return text

    def batch_decode(self, tokens : list[list[int]]):
        "Given a list containing list of tokens, convert it to a list containing strings replacing the tokens with text."
        text = []
        for t in tokens:
            text.append(self.decode(t))
        
        return text
        
    def train_tokenizer(self, raw_data):
        "Given raw data create the token ids by applying the normalization step, pre tokenization step, training step and post tokenization"
        print("Normalizing data...")
        proc_data = ...
        
        print("Pre tokenizing data...")
        proc_data = ...
        
        print("Building vocabulary...")
        self.vocab = ...
        self.vocab_size = ...

        # create mappings between words and ids and ids vs word for encoding and decoding
        print("Building ids to word mappings...")
        self.word_to_ids = {}
        self.ids_to_word = {}
        for i, word in enumerate(self.vocab):
            ...

    def __len__(self):
        "Return the length of the vocabulary"
        return ...
        
    def _normalize(self, raw_data):
        "Applies normalization steps to raw data"
        ## make lower
        proc_data = ...
        
        ## remove accents
        proc_data = ...
        proc_data = ...
        proc_data = ...
        
        ## remove more than one separator into blank space
        proc_data = ...

        return ...

    def _pre_tokenize(self, proc_data):
        "Apply pre tokenization step to normalized data"

        tokens = []
        for t in ...:
            tokens.extend(...)
    
        return tokens
```
"""

    display(Markdown(code))


```python
class Tokenizer:
    def __init__(self):
        # regular expression to get tokens by words and punctuation signs.
        self.re_get_tokens = re.compile(...)
        # regular expression to replace string splitters such as blank space or new lines etc with whatever
        self.re_replace_string_splitters = re.compile(...)
        # regular expression to split by underscore "_"
        self.split_by_underscore = re.compile(....)
        # word to id dictionary
        self.word_to_ids = one
    
    def encode(self, data:str):
        "Convert raw data to corresponding token list"
        assert self.word_to_ids is not None, "Tokenizer has not been train. Call self.train_tokenizer"

        proc_data = ...
        proc_data = ...

        tokens = []
        for token_text in proc_data:
            tokens.append(...)

        return tokens

    def batch_encode(self, data:list):
        "Convert a list containing raw data to a list containing their corresponding token lists."
        tokens = []
        for d in data:
            tokens.append(self.encode(d))

        return tokens

    def decode(self, tokens:list[int]):
        "Given a list of tokens, conver it to a string replacing tokens with text"
        assert self.word_to_ids is not None, "Tokenizer has not been train. Call self.train_tokenizer"
        
        text = ...
        text = ' '.join(text)

        return text

    def batch_decode(self, tokens : list[list[int]]):
        "Given a list containing list of tokens, convert it to a list containing strings replacing the tokens with text."
        text = []
        for t in tokens:
            text.append(self.decode(t))
        
        return text
        
    def train_tokenizer(self, raw_data):
        "Given raw data create the token ids by applying the normalization step, pre tokenization step, training step and post tokenization"
        print("Normalizing data...")
        proc_data = ...
        
        print("Pre tokenizing data...")
        proc_data = ...
        
        print("Building vocabulary...")
        self.vocab = ...
        self.vocab_size = ...

        # create mappings between words and ids and ids vs word for encoding and decoding
        print("Building ids to word mappings...")
        self.word_to_ids = {}
        self.ids_to_word = {}
        for i, word in enumerate(self.vocab):
            ...

    def __len__(self):
        "Return the length of the vocabulary"
        return ...
        
    def _normalize(self, raw_data):
        "Applies normalization steps to raw data"
        ## make lower
        proc_data = ...
        
        ## remove accents
        proc_data = ...
        proc_data = ...
        proc_data = ...
        
        ## remove more than one separator into blank space
        proc_data = ...

        return ...

    def _pre_tokenize(self, proc_data):
        "Apply pre tokenization step to normalized data"

        tokens = []
        for t in ...:
            tokens.extend(...)
    
        return tokens
```


In [37]:
class Tokenizer:
    def __init__(self):
        # regular expression to get tokens by words and punctuation signs.
        self.re_get_tokens = re.compile(r'\w+|[^\w\s]+')
        # regular expression to replace string splitters such as \t \n etc with whatever
        self.re_replace_string_splitters = re.compile(r'\s+')
        # regular expression to split by underscore "_"
        self.split_by_underscore = re.compile(r'_+|[^_]+')
        # word to id dictionary
        self.word_to_ids = None
    
    def encode(self, data:str):
        """Convert raw data to corresponding token list"""
        assert self.word_to_ids is not None, "Tokenizer has not been train. Call self.train_tokenizer"

        proc_data = self._normalize(data)
        proc_data = self._pre_tokenize(proc_data)

        tokens = []
        for token_text in proc_data:
            tokens.append(self.word_to_ids[token_text])

        return tokens

    def batch_encode(self, data:list):
        """Convert a list containing raw data to a list containing their corresponding token lists."""
        tokens = []
        for d in data:
            tokens.append(self.encode(d))

        return tokens

    def decode(self, tokens: List[int]):
        """Given a list of tokens, conver it to a string replacing tokens with text"""
        assert self.word_to_ids is not None, "Tokenizer has not been train. Call self.train_tokenizer"
        
        text = [self.ids_to_word[t] for t in tokens]
        text = ' '.join(text)

        return text

    def batch_decode(self, tokens : List[List[int]]):
        """Given a list containing list of tokens, convert it to a list containing strings replacing the tokens with text."""
        text = []
        for t in tokens:
            text.append(self.decode(t))
        
        return text
        
    def train_tokenizer(self, raw_data):
        """Given raw data create the token ids by applying the normalization step, pre tokenization step, training step and post tokenization"""
        print("Normalizing data...")
        proc_data = self._normalize(raw_data)
        
        print("Pre tokenizing data...")
        proc_data = self._pre_tokenize(proc_data)
        
        print("Building vocabulary...")
        self.vocab = set(proc_data)
        self.vocab_size = len(self.vocab)

        # create mappings between words and ids and ids vs word for encoding and decoding
        print("Building ids to word mappings...")
        self.word_to_ids = {}
        self.ids_to_word = {}
        for i, word in enumerate(self.vocab):
            self.word_to_ids[word] = i
            self.ids_to_word[i] = word

    def __len__(self):
        """Return the length of the vocabulary"""
        return self.vocab_size
        
    def _normalize(self, raw_data):
        """Applies normalization steps to raw data"""
        ## make lower
        proc_data = raw_data.lower()
        
        ## remove accents
        proc_data = unicodedata.normalize("NFD", proc_data)
        proc_data = re.sub(r'[\u0301\u0300\u0302]', '', proc_data)
        proc_data = unicodedata.normalize("NFC", proc_data)
        
        ## remove more than one separator into blank space
        proc_data = self.re_replace_string_splitters.sub(' ', proc_data)

        return proc_data

    def _pre_tokenize(self, proc_data):
        """Apply pre tokenization step to normalized data"""

        tokens = []
        for t in self.split_by_underscore.findall(proc_data):
            tokens.extend(self.re_get_tokens.findall(t))
    
        return tokens

##### Task: Traing your tokenizer on the read data and test it.

* First read data
* Second create an instance of the tokenizer class
* Third call train method on readed data.

In [38]:
if assesment_draw_and_fill:
    code = """
```python
## read data from source to train language model on 
raw_data = ""

## reading dataset with PDF Plumber
with pdfplumber.open(...) as pdf:
    ...

## define tokenizer implementing
tokenizer = ...
tokenizer.train_tokenizer(...)
```
"""

    display(Markdown(code))


```python
## read data from source to train language model on 
raw_data = ""

## reading dataset with PDF Plumber
with pdfplumber.open(...) as pdf:
    ...

## define tokenizer implementing
tokenizer = ...
tokenizer.train_tokenizer(...)
```


In [40]:
## read data from source to train language model on 
raw_data = ""

## reading dataset with PDF Plumber
with pdfplumber.open('../data/texto_pdf.pdf') as pdf:
    for page in pdf.pages:
        raw_data += page.extract_text()

## define tokenizer implementing
tokenizer = Tokenizer()
tokenizer.train_tokenizer(raw_data)

Normalizing data...
Pre tokenizing data...
Building vocabulary...
Building ids to word mappings...


##### Test your tokenizer in the following data.

```python
x = "Las redes neuronales son machine learning."

token_list = tokenizer.encode(x)

print(x)
print(token_list)
```

In [41]:
x = "Las redes neuronales son machine learning."
token_list = tokenizer.encode(x)

print(x)
print(token_list)

Las redes neuronales son machine learning.
[192, 60, 25, 107, 199, 64, 260]


Now detokenize the tokenized_list.

```python
x = tokenizer.decode(token_list)

print(x)
print(token_list)
```

In [42]:
x = tokenizer.decode(token_list)

print(x)
print(token_list)

las redes neuronales son machine learning .
[192, 60, 25, 107, 199, 64, 260]


Compare outputs. See how the point "." mark is given seperate to word `learning`. Why because we convert our list of tokens to list of corresponding words. 

As I mentioned, the final postprocessing step of a language application would be in charge of detecting these kind of things and remove the blankspace between the dot and the word. Modelling this probabilistically through a model is absurd, when is a python deterministic operation. Obviously, with the amount of data language models are trained on, they already know how to do this, because sub work tokenization include blank spaces. But without enough data there is no point in having the token `learning.` and `learning`. 

Again, this depend on the application. This specific example is for language generation.